In [1]:
import matplotlib.pyplot as plt
import numpy as np

import antenna_array as aa
from pattern_measurements import HPBW, FNBW, FSLBW, find_nth_peak, get_lobe, find_main_peak
from utils import wavelength, linear_to_db, db_to_linear
from import_pattern import extract_pattern_data, prepare_data

import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, Checkbox

In [2]:
array = aa.AntennaArray(
    name='Demo Array',
    num_elements=8,
    spacings=np.ones(8) * 0.5,
    weights=np.ones(8)
)

N = 360
theta = np.linspace(-np.pi/2, np.pi/2, N)
frequency = 2.4e9 

pattern_matrix, angles, frequencies = extract_pattern_data()
patch_pattern = prepare_data(pattern_matrix, angles, frequencies, N)[:, 0]
patch_pattern = db_to_linear(patch_pattern - np.max(patch_pattern))

In [3]:
def display_pattern(num_elements = 4, spacing = 0.5, direction=0.0):
    array.num_elements = num_elements
    array.spacings = aa.symmetric_spacing(num_elements, spacing) * wavelength(frequency)
    array.weights = np.ones(num_elements) * 1
    
    beta = aa.beam_direction(array, np.radians(direction))
    print(np.degrees(beta))
    pattern = patch_pattern * np.abs(aa.array_factor(array, frequency, theta, beta))
    # pattern = np.abs(aa.array_factor(array, frequency, theta, beta))
    
    fig, ax = plt.subplots(figsize=(10, 5))
    plt.plot(np.degrees(theta), linear_to_db(pattern), label='Array Pattern')
    _, beam_center = find_main_peak(pattern, theta)
    plt.axvline(np.degrees(beam_center), color='green', linestyle='--', label='Beam Center')    
    plt.axvline(direction, color='red', linestyle='--', label='Beam Direction')

    # print(f'HPBW: {np.degrees(HPBW(pattern, theta))}')
    # print(f'FNBW: {np.degrees(FNBW(pattern, theta))}')
    # print(f'FSLBW: {np.degrees(FSLBW(pattern, theta))}')

    plt.title('Antenna Array Pattern')
    plt.xlabel('Angle (degrees)')
    plt.ylabel('Magnitude (dB)')
    plt.ylim(-50, 0)
    plt.grid()
    plt.legend()
    plt.show()
    

In [4]:
plt.figure(figsize=(10, 6))
interact(
    display_pattern,
    num_elements=IntSlider(min=1, max=16, step=1, value=8, description='Number of Elements'),
    spacing=FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Element Spacing (λ)'),
    direction=FloatSlider(min=-90, max=90, step=5, value=22.5, description='Beam Direction (degrees)'),
    plot=Checkbox(value=True, description='Plot Pattern')
)

<Figure size 1000x600 with 0 Axes>

interactive(children=(IntSlider(value=8, description='Number of Elements', max=16, min=1), FloatSlider(value=0…

<function __main__.display_pattern(num_elements=4, spacing=0.5, direction=0.0)>